In [1]:
from cmipld import *
from pyld import *
import re,json,sys

In [32]:
mip= await CMIPFileUtils.read_file_fs('/Users/daniel.ellis/WIPwork/mip-cmor-tables/JSONLD/scripts/compiled/graph_data.json')
cmip6plus =  await CMIPFileUtils.read_file_fs('/Users/daniel.ellis/WIPwork/CMIP6Plus_CVs/JSONLD/scripts/compiled/graph_data.json')

latest = sum([mip,cmip6plus],[])



In [3]:
# classic id extraction for checks
ids = set(re.findall(r'"@id"\s*:\s*"([^"]+)"', json.dumps(latest)))

# get the triplets
triplets = jsonld.to_rdf(latest)


In [4]:
# grab all the types and ids
type_map = {}
nodes = []
links = []

# clnk = re.compile(r'^[^:]+:(.*?)/')
def clink (s):
    return '/'.join(s.split('/')[:-1])
def prefix (s):
    return s.split(':')[0]

# prefix =  re.compile(r'^[^:]+')

for group in triplets.values():
    for t in group:
        s = str(t)
        if 'literal' not in s and '_:' not in s:
            if 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type' in s:
                type_map[t['subject']['value']] = t['object']['value']
                
                node = dict(id=clink(t['subject']['value']),type=t['object']['value'], origin=prefix(t['subject']['value']))
                if node['origin']!='https':
                    nodes.append(node)
            else:
                try:
                    link = dict(source=clink(t['subject']['value']),target=clink(t['object']['value']),predicate=t['predicate']['value'])
                    links.append(link)
                    
                    # additional origin links
                    src  = link['source']
                    path = re.split(r'[:/]',src)
                    for i in range(1,len(path)):
                        me = path[i-1]
                        n2 = dict(id=me,type='directory',origin=path[0])
                        nodes.append(n2)
                        l2= dict(source=path[i-1],target=path[i],predicate='_')
                        links.append(l2)
                    # dont forget the final node. 
                    me = path[i]
                    n2 = dict(id=me,type='directory-path',origin=path[0])
                    nodes.append(n2)
                    # link up to original
                    l2= dict(source=path[i],target=src,predicate='_')
                    links.append(l2)
                    
                except:
                    # skips out the blank nodes
                    print(t)
                    ...
directories = list(set(triplets.keys()))

In [5]:
# find missing and problem keys 
missing = set(ids) - set(type_map.keys()) - set(directories) 
print(len(missing))

30


In [6]:
from collections import Counter

nodeweights = Counter([i['id'] for i in nodes])
linkweights = Counter([f"{i['source']} -> {i['target']}" for i in links])

for i in nodes:
    i['weight'] = nodeweights[i['id']]
    
for i in links: 
    i['weight'] = linkweights[f"{i['source']} -> {i['target']}"]


unodes = list(dict([(i['id'],i) for i in nodes]).values())
ulinks = list(dict([((i['source'],i['target'],i['predicate']),i) for i in links]).values())




In [19]:
tmap = dict([[v,clink(k)] for k,v in type_map.items()])

In [68]:
# graphs = filter(lambda x: 'graph' in x['@type'],latest)

# list(graphs)

# since the types are defined by the vocab
id2vocab = {v: k.replace('mip:','') for k, v in tmap.items() if 'https' not in v}

id2vocab

{'mip-cmor-tables:auxillary/frequency': 'frequency',
 'mip-cmor-tables:auxillary/mip-era': 'mip-era',
 'mip-cmor-tables:auxillary/native-nominal-resolution': 'native-nominal-resolution',
 'mip-cmor-tables:auxillary/product': 'product',
 'mip-cmor-tables:auxillary/realm': 'realm',
 'mip-cmor-tables:auxillary/source-type': 'source-type',
 'mip-cmor-tables:grid/label': 'grid-label',
 'mip-cmor-tables:license': 'license',
 'mip-cmor-tables:organisations/consortia': 'consortium',
 'mip-cmor-tables:organisations/institutions': 'institution',
 'mip-cmor-tables:organisations/institutions/location': 'institution:location',
 'cmip6plus:activity': 'activity-id',
 'cmip6plus:core': 'index',
 'cmip6plus:experiment/id': 'experiment-id',
 'cmip6plus:experiment/sub-id': 'sub-experiment-id',
 'cmip6plus:model/component': 'model-component',
 'cmip6plus:source/id': 'source-id'}

In [20]:
with open('network.json','w') as f:
    json.dump(dict(nodes=unodes,links=ulinks,types=tmap,vocab=id2vocab),f,indent=2)

In [9]:
# import xml.etree.ElementTree as ET

# # Create the root element for GraphML
# graphml = ET.Element('graphml', xmlns="http://graphml.graphdrawing.org/xmlns")
# # Create the graph element
# graph = ET.SubElement(graphml, 'graph', id="G", edgedefault="directed")

# # # Create a key element for edge data (predicate)
# # key = ET.SubElement(graphml, 'key', id="predicate", for="edge", attr.name="predicate", attr.type="string")


# for node in nodes:
#     ET.SubElement(graph, 'node', **node)

# # Add edges
# for entry in links:

#     edge = ET.SubElement(graph, 'edge', **entry)
#     # Add predicate as edge data
#     # data_element = ET.SubElement(edge, 'data', key="predicate")
#     # data_element.text = predicate

# # Create an ElementTree object and write to a GraphML file
# tree = ET.ElementTree(graphml)
# tree.write('graph.graphml', xml_declaration=True, encoding='utf-8', method='xml')

# print("GraphML file saved as 'graph.graphml'")


In [10]:
linkweights

Counter({'mip-cmor-tables:organisations/institutions -> mip-cmor-tables:organisations/institutions/location': 72,
         'mip-cmor-tables -> organisations': 72,
         'organisations -> institutions': 72,
         'institutions -> mip-cmor-tables:organisations/institutions': 72,
         'cmip6plus -> model': 58,
         'model -> component': 58,
         'component -> cmip6plus:model/component': 58,
         'cmip6plus -> experiment': 56,
         'experiment -> id': 56,
         'id -> cmip6plus:experiment/id': 56,
         'cmip6plus -> source': 54,
         'source -> id': 54,
         'id -> cmip6plus:source/id': 54,
         'cmip6plus:source/id -> cmip6plus:model/component': 36,
         'cmip6plus:model/component -> mip-cmor-tables:auxillary/native-nominal-resolution': 29,
         'cmip6plus:model/component -> mip-cmor-tables:auxillary/realm': 29,
         'cmip6plus:experiment/id -> cmip6plus:activity': 28,
         'cmip6plus:experiment/id -> cmip6plus:experiment/sub-id

In [63]:
print(json.dumps(latest))

[{"@id": "https://github.com/PCMDI/mip-cmor-tables/JSONLD/license", "@type": "graph", "@context": {"@vocab": "license:", "@base": "https://github.com/PCMDI/mip-cmor-tables"}, "ldroot": "/JSONLD/license", "@graph": [{"@id": "mip-cmor-tables:license/cc0-1.0", "@type": "mip:license", "license:conditions": "The CMIP6Plus model data linking to this entry is licensed under a Creative Commons CC0 1.0 Universal Public Domain Dedication (https://creativecommons.org/publicdomain/zero/1.0/). Consult https://pcmdi.llnl.gov/CMIP6/TermsOfUse for terms of use governing CMIP6Plus output, including citation requirements and proper acknowledgment. The data producers and data providers make no warranty, either express or implied, including, but not limited to, warranties of merchantability and fitness for a particular purpose. All liabilities arising from the supply of the information (including any liability arising in negligence) are excluded to the fullest extent permitted by law.", "license:kind": "C